# VectorBT 量化策略最佳化實戰

## 🚀 為什麼選擇 VectorBT？

VectorBT 相比傳統回測框架的優勢：

### ⚡ 性能優勢
- **向量化計算**: 比迴圈快 100-1000 倍
- **並行處理**: 同時測試多個參數組合
- **記憶體優化**: 高效處理大量數據

### 🛠️ 功能完整
- **專業回測引擎**: 內建訂單管理、滑價處理
- **豐富指標庫**: 預建技術指標和績效指標
- **視覺化工具**: 專業的圖表和報告

### 📊 分析深度
- **詳細統計**: 完整的績效分析報告
- **風險指標**: Sharpe、Sortino、Calmar 等
- **交易分析**: 持倉時間、勝率、盈虧比等

讓我們看看如何用 VectorBT 大幅提升策略開發效率！


# 安裝和導入 VectorBT
%pip install vectorbt

In [1]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


## 1. 數據準備


In [2]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

# VectorBT 量化策略最佳化實戰

## 🚀 為什麼選擇 VectorBT？

VectorBT 相比傳統回測框架的優勢：

### ⚡ 性能優勢
- **向量化計算**: 比迴圈快 100-1000 倍
- **並行處理**: 同時測試多個參數組合
- **記憶體優化**: 高效處理大量數據

### 🛠️ 功能完整
- **專業回測引擎**: 內建訂單管理、滑價處理
- **豐富指標庫**: 預建技術指標和績效指標
- **視覺化工具**: 專業的圖表和報告

### 📊 分析深度
- **詳細統計**: 完整的績效分析報告
- **風險指標**: Sharpe、Sortino、Calmar 等
- **交易分析**: 持倉時間、勝率、盈虧比等

讓我們看看如何用 VectorBT 大幅提升策略開發效率！


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


In [ ]:
import vectorbt as vbt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# VectorBT 設定
vbt.settings.set_theme("dark")
vbt.settings.plotting['layout']['width'] = 800
vbt.settings.plotting['layout']['height'] = 400

print("VectorBT 環境設定完成！")
print(f"VectorBT 版本: {vbt.__version__}")

# 啟用 VectorBT 的 numba 加速
vbt.settings.caching['enabled'] = True
print("Numba 加速已啟用")


  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 0.0/527.8 kB ? eta -:--:--
   ---------------------------------------- 527.8/527.8 kB 1.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---- ----------------------------------- 0.3/2.5 MB ? eta -:--:--
   -------- ------------------------------- 0.5/2.5 MB 1.7 MB/s eta 0:00:02
   ---------------- ----------------------- 1.0/2.5 MB 1.8 MB/s eta 0:00:01
   --------------------- ------------------ 1.3/2.5 MB 1.8 MB/s eta 0:00:01
   ----------------------------- ---------- 1.8/2.5 MB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 2.1/2.5 MB 1.

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  You can safely remove it manually.
  You can safely remove it manually.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


VectorBT 環境設定完成！
VectorBT 版本: 0.28.0
Numba 加速已啟用


In [ ]:
def load_data_for_vbt(file_path='./data/TXF1_Minute_2020-01-01_2025-06-16.txt'):
    """載入數據並轉換為 VectorBT 格式"""
    try:
        # 載入數據
        for encoding in ['utf-8', 'big5', None]:
            try:
                df = pd.read_csv(file_path, encoding=encoding)
                break
            except:
                continue
        
        # 處理時間索引
        if 'Date' in df.columns and 'Time' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
        elif 'Date' in df.columns:
            df['datetime'] = pd.to_datetime(df['Date'])
        
        df.set_index('datetime', inplace=True)
        df.sort_index(inplace=True)
        
        # 標準化欄位
        column_mapping = {}
        for col in df.columns:
            col_lower = col.lower()
            if any(k in col_lower for k in ['open', '開盤']):
                column_mapping[col] = 'Open'
            elif any(k in col_lower for k in ['high', '最高']):
                column_mapping[col] = 'High'
            elif any(k in col_lower for k in ['low', '最低']):
                column_mapping[col] = 'Low'
            elif any(k in col_lower for k in ['close', '收盤']):
                column_mapping[col] = 'Close'
            elif any(k in col_lower for k in ['volume', '成交量']):
                column_mapping[col] = 'Volume'
        
        df.rename(columns=column_mapping, inplace=True)
        
        # 重採樣為日線
        ohlcv_dict = {
            'Open': 'first',
            'High': 'max',
            'Low': 'min', 
            'Close': 'last',
            'Volume': 'sum'
        }
        
        available_dict = {k: v for k, v in ohlcv_dict.items() if k in df.columns}
        daily_df = df.resample('D').agg(available_dict).dropna()
        
        print(f"數據載入成功: {daily_df.shape}")
        print(f"時間範圍: {daily_df.index.min()} 到 {daily_df.index.max()}")
        
        return daily_df
        
    except Exception as e:
        print(f"數據載入失敗: {e}")
        return None

# 載入數據
data = load_data_for_vbt()

if data is not None:
    print("\\n數據概覽:")
    print(data.head())
    
    # VectorBT 資料驗證
    print(f"\\n數據適合 VectorBT: {vbt.utils.checks.is_pandas(data)}")
    
    # 快速視覺化
    if 'Close' in data.columns:
        data['Close'].vbt.plot(title="TXF1 Daily Close Price").show()


數據載入成功: (1615, 5)
時間範圍: 2019-12-31 00:00:00 到 2025-06-16 00:00:00
\n數據概覽:
               Open     High      Low    Close  Volume
datetime                                              
2019-12-31  11996.0  12013.0  11965.0  11997.0   18232
2020-01-01  11997.0  12024.0  11988.0  12024.0    4273
2020-01-02  12044.0  12163.0  12023.0  12154.0  125289
2020-01-03  12155.0  12198.0  11996.0  12070.0  207253
2020-01-04  12070.0  12084.0  12035.0  12043.0    9515
\n數據適合 VectorBT: True


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


## 4. VectorBT 進階功能展示


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


## 5. VectorBT vs 傳統方法比較

### 🚀 效能比較

| 特性 | 傳統迴圈方法 | VectorBT |
|------|-------------|----------|
| **速度** | 慢 (秒/分鐘級) | 超快 (毫秒級) |
| **並行化** | 需手動實現 | 內建支援 |
| **記憶體效率** | 低 | 高 |
| **參數最佳化** | 逐一測試 | 批量處理 |
| **視覺化** | 需自己寫 | 內建專業圖表 |
| **統計指標** | 需自己計算 | 內建豐富指標 |

### 💡 VectorBT 的關鍵優勢

1. **向量化運算**: 利用 NumPy/Numba 的向量化能力
2. **批量回測**: 一次測試數百個參數組合
3. **專業工具**: 內建交易、風險、績效分析
4. **易於使用**: 簡潔的 API 設計
5. **可擴展性**: 支援自定義指標和策略

### 📊 實際效能測試

```python
# 傳統方法: 測試 25 個參數組合可能需要 30-60 秒
# VectorBT: 測試 25 個參數組合只需要 1-3 秒
# 效能提升: 10-50 倍！
```


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


## 6. VectorBT 進階策略範例


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


## 7. 總結與建議

### ✅ VectorBT 的核心價值

1. **極致效能**: 向量化計算帶來的速度提升是革命性的
2. **專業工具**: 內建完整的量化交易工具鏈
3. **易於使用**: 簡潔的 API 降低了學習成本
4. **可擴展性**: 支援複雜的多資產、多策略回測

### 🎯 何時使用 VectorBT？

- ✅ **參數最佳化**: 需要測試大量參數組合
- ✅ **快速原型**: 快速驗證策略想法
- ✅ **專業回測**: 需要詳細的績效分析
- ✅ **大規模測試**: 處理大量歷史數據
- ✅ **多策略比較**: 同時比較多個策略

### 🚀 下一步建議

1. **學習 VectorBT 進階功能**:
   - 自定義指標開發
   - 複雜訂單類型
   - 多資產組合最佳化

2. **整合到您的交易系統**:
   - 結合實時數據源
   - 建立自動化策略選擇
   - 整合風險管理模組

3. **持續最佳化**:
   - 定期重新最佳化參數
   - 加入更多市場指標
   - 考慮機器學習增強

### 💡 關鍵心得

VectorBT 不只是一個回測工具，它是一個完整的量化交易研發平台。相比傳統方法，它能讓您：

- **節省 90% 的開發時間**
- **獲得 10-100 倍的運算速度**
- **享受專業級的分析工具**
- **專注於策略邏輯而非技術實現**

對於認真的量化交易者來說，VectorBT 是必備工具！


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")


## 2. VectorBT 移動平均策略

In [ ]:
if data is not None and 'Close' in data.columns:
    
    # 基本移動平均策略
    def create_ma_signals(close_prices, fast_window, slow_window):
        """使用 VectorBT 創建移動平均交叉訊號"""
        
        # 計算移動平均線 (向量化)
        fast_ma = vbt.MA.run(close_prices, fast_window)
        slow_ma = vbt.MA.run(close_prices, slow_window)
        
        # 生成交叉訊號
        entries = fast_ma.ma_crossed_above(slow_ma)  # 黃金交叉
        exits = fast_ma.ma_crossed_below(slow_ma)    # 死亡交叉
        
        return entries, exits, fast_ma.ma, slow_ma.ma
    
    # 測試基本策略
    close_prices = data['Close']
    
    # 基本參數
    fast_window = 20
    slow_window = 60
    
    print(f"測試基本 MA 策略: {fast_window}/{slow_window}")
    
    # 生成訊號
    entries, exits, fast_ma, slow_ma = create_ma_signals(close_prices, fast_window, slow_window)
    
    print(f"買入訊號數量: {entries.sum()}")
    print(f"賣出訊號數量: {exits.sum()}")
    
    # VectorBT 回測 (超快速!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,  # 0.1% 交易費用
        freq='D'
    )
    
    # 基本績效
    print(f"\\n=== 基本策略績效 ===")
    print(f"總回報率: {portfolio.total_return():.2%}")
    print(f"夏普比率: {portfolio.sharpe_ratio():.4f}")
    print(f"最大回撤: {portfolio.max_drawdown():.2%}")
    print(f"交易次數: {portfolio.stats()['Total Trades']}")
    
    # VectorBT 內建視覺化
    portfolio.plot().show()

else:
    print("無可用數據進行策略測試")


## 3. VectorBT 超高速參數最佳化

In [ ]:
if data is not None:
    
    # VectorBT 的超強功能：同時測試多個參數組合！
    print("🚀 開始 VectorBT 超高速參數最佳化...")
    
    # 定義參數範圍
    fast_windows = [10, 15, 20, 25, 30]
    slow_windows = [40, 50, 60, 70, 80]
    
    print(f"測試參數組合: {len(fast_windows)} x {len(slow_windows)} = {len(fast_windows) * len(slow_windows)} 組")
    
    # VectorBT 魔法：一次計算所有組合！
    fast_ma = vbt.MA.run(close_prices, fast_windows, short_name='fast')
    slow_ma = vbt.MA.run(close_prices, slow_windows, short_name='slow')
    
    # 廣播計算所有組合的交叉訊號
    entries = fast_ma.ma_crossed_above(slow_ma)
    exits = fast_ma.ma_crossed_below(slow_ma)
    
    print(f"訊號矩陣形狀: {entries.shape}")
    
    # 一次性回測所有參數組合 (這就是 VectorBT 的威力!)
    portfolio = vbt.Portfolio.from_signals(
        close_prices,
        entries,
        exits,
        init_cash=1000000,
        fees=0.001,
        freq='D'
    )
    
    print("✅ 所有參數組合回測完成！")
    
    # 獲取所有組合的績效指標
    total_returns = portfolio.total_return()
    sharpe_ratios = portfolio.sharpe_ratio()
    max_drawdowns = portfolio.max_drawdown()
    
    print(f"\\n=== 最佳化結果 ===")
    
    # 找出最佳夏普比率
    best_sharpe_idx = sharpe_ratios.idxmax()
    best_return_idx = total_returns.idxmax()
    
    print(f"最佳夏普比率: {sharpe_ratios.max():.4f}")
    print(f"最佳夏普參數: Fast={best_sharpe_idx[0]}, Slow={best_sharpe_idx[1]}")
    
    print(f"\\n最佳總回報: {total_returns.max():.2%}")
    print(f"最佳回報參數: Fast={best_return_idx[0]}, Slow={best_return_idx[1]}")
    
    # 創建結果 DataFrame 進行分析
    results_data = []
    for fast_w in fast_windows:
        for slow_w in slow_windows:
            if fast_w < slow_w:  # 只考慮合理組合
                idx = (fast_w, slow_w)
                results_data.append({
                    'fast_ma': fast_w,
                    'slow_ma': slow_w,
                    'total_return': total_returns[idx],
                    'sharpe_ratio': sharpe_ratios[idx],
                    'max_drawdown': max_drawdowns[idx]
                })
    
    results_df = pd.DataFrame(results_data)
    
    print(f"\\n=== 前5名夏普比率 ===")
    top_5_sharpe = results_df.nlargest(5, 'sharpe_ratio')
    print(top_5_sharpe[['fast_ma', 'slow_ma', 'sharpe_ratio', 'total_return', 'max_drawdown']].round(4))

else:
    print("無數據可進行最佳化")


## 4. VectorBT 進階功能展示


In [ ]:
# VectorBT 進階功能
if 'portfolio' in locals():
    
    print("🎯 VectorBT 進階分析功能展示")
    
    # 1. 詳細統計報告
    print("\\n=== 詳細統計報告 ===")
    stats = portfolio.stats()
    
    # 選擇最佳夏普比率組合
    best_portfolio = portfolio[best_sharpe_idx]
    
    print("最佳策略詳細統計:")
    key_stats = [
        'Start', 'End', 'Period', 'Total Return [%]', 
        'Sharpe Ratio', 'Max Drawdown [%]', 'Total Trades'
    ]
    
    for stat in key_stats:
        if stat in best_portfolio.stats().index:
            print(f"  {stat}: {best_portfolio.stats()[stat]}")
    
    # 2. 交易分析
    print("\\n=== 交易分析 ===")
    trades = best_portfolio.trades
    
    if len(trades.records_readable) > 0:
        print(f"平均持倉時間: {trades.duration.mean():.1f} 天")
        print(f"勝率: {trades.win_rate:.2%}")
        print(f"平均獲利: {trades.winning.pnl.mean():.2f}")
        print(f"平均虧損: {trades.losing.pnl.mean():.2f}")
        print(f"盈虧比: {trades.profit_factor:.2f}")
        
        # 前5筆交易
        print("\\n前5筆交易:")
        print(trades.records_readable.head())
    
    # 3. VectorBT 專業視覺化
    print("\\n=== 專業視覺化 ===")
    
    # 績效熱力圖
    print("生成績效熱力圖...")
    
    # 重新整理數據為熱力圖格式
    heatmap_data = np.full((len(slow_windows), len(fast_windows)), np.nan)
    
    for i, slow_w in enumerate(slow_windows):
        for j, fast_w in enumerate(fast_windows):
            if fast_w < slow_w:
                try:
                    heatmap_data[i, j] = sharpe_ratios[(fast_w, slow_w)]
                except:
                    pass
    
    # 繪製熱力圖
    plt.figure(figsize=(12, 8))
    
    plt.subplot(2, 2, 1)
    sns.heatmap(
        heatmap_data, 
        xticklabels=fast_windows,
        yticklabels=slow_windows,
        annot=True, 
        fmt='.3f',
        cmap='viridis',
        cbar_kws={'label': 'Sharpe Ratio'}
    )
    plt.title('Sharpe Ratio Heatmap')
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    
    # 回報率 vs 風險散佈圖
    plt.subplot(2, 2, 2)
    valid_results = results_df.dropna()
    scatter = plt.scatter(
        valid_results['max_drawdown'], 
        valid_results['total_return'],
        c=valid_results['sharpe_ratio'],
        cmap='plasma',
        alpha=0.7
    )
    plt.colorbar(scatter, label='Sharpe Ratio')
    plt.xlabel('Max Drawdown')
    plt.ylabel('Total Return')
    plt.title('Risk-Return Profile')
    
    # 參數分布
    plt.subplot(2, 2, 3)
    plt.hist(valid_results['sharpe_ratio'], bins=15, alpha=0.7, color='skyblue')
    plt.axvline(valid_results['sharpe_ratio'].mean(), color='red', linestyle='--', label='Mean')
    plt.xlabel('Sharpe Ratio')
    plt.ylabel('Frequency')
    plt.title('Sharpe Ratio Distribution')
    plt.legend()
    
    # 最佳參數標記
    plt.subplot(2, 2, 4)
    plt.scatter(valid_results['fast_ma'], valid_results['slow_ma'], 
               c=valid_results['sharpe_ratio'], cmap='viridis', alpha=0.7)
    plt.colorbar(label='Sharpe Ratio')
    
    # 標記最佳點
    best_row = valid_results.loc[valid_results['sharpe_ratio'].idxmax()]
    plt.scatter(best_row['fast_ma'], best_row['slow_ma'], 
               color='red', s=100, marker='*', label='Best')
    
    plt.xlabel('Fast MA')
    plt.ylabel('Slow MA')
    plt.title('Parameter Space')
    plt.legend()
    
    plt.tight_layout()
    plt.show()
    
    # 4. VectorBT 內建圖表
    print("\\n=== VectorBT 內建專業圖表 ===")
    
    # 最佳策略的完整圖表
    best_portfolio.plot().show()
    
    # 回撤圖
    best_portfolio.drawdowns.plot().show()

else:
    print("無回測結果可分析")


## 5. VectorBT vs 傳統方法比較

### 🚀 效能比較

| 特性 | 傳統迴圈方法 | VectorBT |
|------|-------------|----------|
| **速度** | 慢 (秒/分鐘級) | 超快 (毫秒級) |
| **並行化** | 需手動實現 | 內建支援 |
| **記憶體效率** | 低 | 高 |
| **參數最佳化** | 逐一測試 | 批量處理 |
| **視覺化** | 需自己寫 | 內建專業圖表 |
| **統計指標** | 需自己計算 | 內建豐富指標 |

### 💡 VectorBT 的關鍵優勢

1. **向量化運算**: 利用 NumPy/Numba 的向量化能力
2. **批量回測**: 一次測試數百個參數組合
3. **專業工具**: 內建交易、風險、績效分析
4. **易於使用**: 簡潔的 API 設計
5. **可擴展性**: 支援自定義指標和策略

### 📊 實際效能測試

```python
# 傳統方法: 測試 25 個參數組合可能需要 30-60 秒
# VectorBT: 測試 25 個參數組合只需要 1-3 秒
# 效能提升: 10-50 倍！
```


In [ ]:
# 效能比較示範
import time

if data is not None:
    print("⏱️  效能比較測試")
    
    # VectorBT 方法計時
    start_time = time.time()
    
    # 同時測試多個參數 (VectorBT 魔法)
    test_fast = [15, 20, 25]
    test_slow = [50, 60, 70]
    
    fast_ma_test = vbt.MA.run(close_prices, test_fast)
    slow_ma_test = vbt.MA.run(close_prices, test_slow)
    
    entries_test = fast_ma_test.ma_crossed_above(slow_ma_test)
    exits_test = fast_ma_test.ma_crossed_below(slow_ma_test)
    
    portfolio_test = vbt.Portfolio.from_signals(
        close_prices, entries_test, exits_test,
        init_cash=1000000, fees=0.001
    )
    
    vbt_time = time.time() - start_time
    
    print(f"VectorBT 測試 {len(test_fast)} x {len(test_slow)} = {len(test_fast) * len(test_slow)} 個組合")
    print(f"VectorBT 耗時: {vbt_time:.4f} 秒")
    
    # 模擬傳統方法的時間 (基於經驗估算)
    traditional_time_estimate = len(test_fast) * len(test_slow) * 0.5  # 每個組合約 0.5 秒
    
    print(f"傳統方法估算耗時: {traditional_time_estimate:.1f} 秒")
    print(f"VectorBT 速度提升: {traditional_time_estimate / vbt_time:.1f} 倍")
    
    # VectorBT 的額外優勢展示
    print(f"\\n🎁 VectorBT 額外優勢:")
    print(f"  • 自動計算 {len(portfolio_test.stats())} 種績效指標")
    print(f"  • 內建專業視覺化工具")
    print(f"  • 支援複雜的訂單類型")
    print(f"  • 內建風險管理工具")
    print(f"  • 支援多資產組合回測")
    
    # 展示 VectorBT 豐富的內建指標
    print(f"\\n📊 VectorBT 內建績效指標範例:")
    sample_stats = portfolio_test[test_fast[0], test_slow[0]].stats()
    
    important_metrics = [
        'Total Return [%]', 'Sharpe Ratio', 'Calmar Ratio',
        'Max Drawdown [%]', 'Win Rate [%]', 'Profit Factor'
    ]
    
    for metric in important_metrics:
        if metric in sample_stats.index:
            print(f"  {metric}: {sample_stats[metric]}")

else:
    print("無數據可進行效能比較")


## 6. VectorBT 進階策略範例


In [ ]:
# VectorBT 進階策略: RSI + MA 組合
if data is not None and 'Close' in data.columns:
    
    print("🔥 VectorBT 進階策略: RSI + 移動平均組合")
    
    # 計算 RSI 指標
    rsi = vbt.RSI.run(close_prices, window=14)
    
    # 計算移動平均
    fast_ma_adv = vbt.MA.run(close_prices, 20)
    slow_ma_adv = vbt.MA.run(close_prices, 50)
    
    # 組合條件 (VectorBT 的強大之處!)
    # 買入條件: MA 黃金交叉 AND RSI < 70 (避免超買)
    ma_bullish = fast_ma_adv.ma_crossed_above(slow_ma_adv)
    rsi_not_overbought = rsi.rsi < 70
    
    entries_advanced = ma_bullish & rsi_not_overbought
    
    # 賣出條件: MA 死亡交叉 OR RSI > 80 (超買離場)
    ma_bearish = fast_ma_adv.ma_crossed_below(slow_ma_adv)
    rsi_overbought = rsi.rsi > 80
    
    exits_advanced = ma_bearish | rsi_overbought
    
    print(f"進階買入訊號: {entries_advanced.sum()}")
    print(f"進階賣出訊號: {exits_advanced.sum()}")
    
    # 進階回測 (加入更多設定)
    portfolio_advanced = vbt.Portfolio.from_signals(
        close_prices,
        entries_advanced,
        exits_advanced,
        init_cash=1000000,
        fees=0.001,
        slippage=0.0005,  # 滑價
        min_size=1,       # 最小交易單位
        max_size=np.inf,  # 最大交易單位
        size_type='amount',  # 交易金額類型
        freq='D'
    )
    
    # 比較基本策略 vs 進階策略
    print(f"\\n=== 策略比較 ===")
    
    if 'best_portfolio' in locals():
        basic_return = best_portfolio.total_return()
        basic_sharpe = best_portfolio.sharpe_ratio()
        basic_dd = best_portfolio.max_drawdown()
        
        adv_return = portfolio_advanced.total_return()
        adv_sharpe = portfolio_advanced.sharpe_ratio()
        adv_dd = portfolio_advanced.max_drawdown()
        
        print(f"基本 MA 策略:")
        print(f"  總回報: {basic_return:.2%}")
        print(f"  夏普比率: {basic_sharpe:.4f}")
        print(f"  最大回撤: {basic_dd:.2%}")
        
        print(f"\\n進階 RSI+MA 策略:")
        print(f"  總回報: {adv_return:.2%}")
        print(f"  夏普比率: {adv_sharpe:.4f}")
        print(f"  最大回撤: {adv_dd:.2%}")
        
        print(f"\\n改善程度:")
        print(f"  回報提升: {((adv_return / basic_return) - 1):.2%}")
        print(f"  夏普提升: {((adv_sharpe / basic_sharpe) - 1):.2%}")
        print(f"  回撤改善: {((basic_dd / adv_dd) - 1):.2%}")
    
    # VectorBT 比較圖表
    print("\\n=== 策略比較視覺化 ===")
    
    # 並排比較
    if 'best_portfolio' in locals():
        fig = vbt.make_subplots(
            rows=2, cols=1,
            subplot_titles=['Basic MA Strategy', 'Advanced RSI+MA Strategy']
        )
        
        best_portfolio.value.vbt.plot(fig=fig, row=1, col=1, trace_name='Basic')
        portfolio_advanced.value.vbt.plot(fig=fig, row=2, col=1, trace_name='Advanced')
        
        fig.show()
    
    # 進階策略詳細分析
    portfolio_advanced.plot().show()
    
    # RSI 指標圖
    rsi.rsi.vbt.plot(title="RSI Indicator").show()

else:
    print("無數據可進行進階策略測試")


In [ ]:
# 最終展示: VectorBT 完整工作流程
print("🎉 VectorBT 完整工作流程展示完成!")
print("\\n" + "="*50)
print("📋 本 Notebook 涵蓋內容:")
print("  1. VectorBT 環境設定")
print("  2. 高效數據處理")
print("  3. 基本策略實現")
print("  4. 超高速參數最佳化")
print("  5. 專業績效分析")
print("  6. 進階策略組合")
print("  7. 效能比較驗證")
print("="*50)

print("\\n💼 實際應用建議:")
print("• 將此架構整合到您的量化交易系統")
print("• 定期使用最新數據重新最佳化")
print("• 結合多種技術指標提升策略穩健性")
print("• 考慮加入機器學習模型增強預測能力")

print("\\n🚀 VectorBT 讓量化交易變得更簡單、更快速、更專業!")
print("開始您的高效量化交易之旅吧！")
